In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import time
import os
from pathlib import Path
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.utils.tensorboard import SummaryWriter

import sys
sys.path.append("..")

# dataset manager
from dcase2020.datasetManager import DESEDManager
from dcase2020.datasets import DESEDDataset

# utility function & metrics & augmentation
from metric_utils.metrics import FScore, BinaryAccuracy
from dcase2020_task4.util.utils import get_datetime, reset_seed

# models
from dcase2020_task4.baseline.models import WeakBaseline

In [3]:
# ==== set the log ====
import logging
import logging.config
from dcase2020.util.log import DEFAULT_LOGGING
logging.config.dictConfig(DEFAULT_LOGGING)
log = logging.getLogger(__name__)

In [4]:
# ==== reset the seed for reproductability ====
reset_seed(1234)

# Loading the dataset

In [8]:
# ==== load the dataset ====
# desed_metadata_root = "../dataset/DESED/dataset/metadata"
# desed_audio_root = "../dataset/DESED/dataset/audio"
desed_metadata_root = os.path.join("e:/", "Corpus", "dcase2020", "DESED", "dataset", "metadata")
desed_audio_root = os.path.join("e:/", "Corpus", "dcase2020", "DESED", "dataset", "audio")

manager = DESEDManager(
    desed_metadata_root, desed_audio_root,
    sampling_rate = 22050,
    validation_ratio=0.2,
    from_disk=True,
    verbose = 2
)

DEBUG --- datasetManager.__init__ >>> e:/Corpus\dcase2020\DESED\dataset\audio\dcase2020_dataset_22050.hdf5
INFO --- datasetManager._load_metadata >>> Reading metadata: e:/Corpus\dcase2020\DESED\dataset\metadata\train\weak.tsv
INFO --- datasetManager._load_metadata >>> Reading metadata: e:/Corpus\dcase2020\DESED\dataset\metadata\train\unlabel_in_domain.tsv
INFO --- datasetManager._load_metadata >>> Reading metadata: e:/Corpus\dcase2020\DESED\dataset\metadata\train\synthetic20.tsv



In [9]:
manager.add_subset("weak")

INFO --- datasetManager.add_subset >>> Loading dataset: train, subset: weak
Loading dataset: train, subset: weak
from disk:  True
<HDF5 file "dcase2020_dataset_22050.hdf5" (mode r)>
DESED\dataset\audio\train
DESED\dataset\audio\train\weak


In [10]:
manager.split_train_validation()

INFO --- datasetManager.split_train_validation >>> Creating new train / validation split
INFO --- datasetManager.split_train_validation >>> validation ratio : 0.2


In [11]:
augments = [
    # signal_augmentation.Noise(0.5, target_snr=15),
    # signal_augmentation.RandomTimeDropout(0.5, dropout=0.2)
]

train_dataset = DESEDDataset(manager, train=True, val=False, augments=augments, cached=True)
val_dataset = DESEDDataset(manager, train=False, val=True, augments=[], cached=True)

In [12]:
len(train_dataset.filenames), len(val_dataset.filenames)

(1097, 243)

# Prepare training

## model

In [14]:
model = WeakBaseline()
model.cuda()

In [16]:
from torchsummaryX import summary
input_tensor = torch.zeros((1, 64, 431), dtype=torch.float)
input_tensor = input_tensor.cuda()

s = summary(model, input_tensor)


                               Kernel Shape      Output Shape  Params  \
Layer                                                                   
0_features.0.Conv2d_0         [1, 32, 3, 3]  [1, 32, 64, 431]   320.0   
1_features.0.MaxPool2d_1                  -  [1, 32, 16, 215]       -   
2_features.0.BatchNorm2d_2             [32]  [1, 32, 16, 215]    64.0   
3_features.0.Dropout2d_3                  -  [1, 32, 16, 215]       -   
4_features.0.ReLU6_4                      -  [1, 32, 16, 215]       -   
5_features.1.Conv2d_0        [32, 32, 3, 3]  [1, 32, 16, 215]  9.248k   
6_features.1.MaxPool2d_1                  -   [1, 32, 4, 107]       -   
7_features.1.BatchNorm2d_2             [32]   [1, 32, 4, 107]    64.0   
8_features.1.Dropout2d_3                  -   [1, 32, 4, 107]       -   
9_features.1.ReLU6_4                      -   [1, 32, 4, 107]       -   
10_features.2.Conv2d_0       [32, 32, 3, 3]   [1, 32, 4, 107]  9.248k   
11_features.2.MaxPool2d_1                 -    [1, 

## Training parameters (crit & callbacks & loaders & metrics)

In [23]:
# training parameters
nb_epochs = 100
batch_size = 32
nb_batch = len(train_dataset) // batch_size

# criterion & optimizers
criterion = nn.BCEWithLogitsLoss(reduction="mean")

optimizers = torch.optim.Adam(model.parameters(), lr=0.003)

# callbacks
callbacks = []

# tensorboard
title = "WeakBaseline_%s" % (get_datetime())
tensorboard = SummaryWriter(log_dir=Path("../tensorboard/%s" % title), comment="weak baseline")

# loaders
training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Metrics
binacc_func = BinaryAccuracy()

## Training functions

In [41]:
def train(epoch: int):
    start_time = time.time()
    binacc_func.reset()
    model.train()
    print("") # <-- Force new line
    
    for i, (X_weak, y_weak) in enumerate(training_loader):
        # The DESEDDataset return a list of ground truth depending on the selecting option.
        # If weak and strong ground truth are selected, the list order is [WEAK, STRONG]
        # here there is only one [WEAK]
        X_weak, y_weak = X_weak.cuda().float(), y_weak[0].cuda().float()

        logits = model(X_weak)
        
        loss = criterion(logits, y_weak)
        
        # calc metrics
        pred = F.sigmoid(logits)
        binacc = binacc_func(pred, y_weak)
        
        # back propagation
        optimizers.zero_grad()
        loss.backward()
        optimizers.step()
        
        # logs
        print("Epoch {}, {:d}% \t loss: {:.4e} - acc: {:.4e} - took {:.2f}s".format(
            epoch + 1,
            int(100 * (i + 1) / nb_batch),
            loss.item(),
            binacc,
            time.time() - start_time
        ), end="\r")
        
    # tensorboard logs
    tensorboard.add_scalar("train/loss", loss.item(), epoch)
    tensorboard.add_scalar("train/acc", binacc, epoch)
    

In [42]:
def val(epoch):
    binacc_func.reset()
    model.train()
    print("") # <-- Force new line
    
    for i, (X_weak, y_weak) in enumerate(val_loader):
        X_weak, y_weak = X_weak.cuda().float(), y_weak[0].cuda().float()

        logits = model(X_weak)
        
        loss = criterion(logits, y_weak)
        
        # calc metrics
        pred = F.sigmoid(logits)
        binacc = binacc_func(pred, y_weak)

        # logs
        print("validation \t val_loss: {:.4e} - val_acc: {:.4e}".format(
            loss.item(),
            binacc,
        ), end="\r")

    # tensorboard logs
    tensorboard.add_scalar("val/loss", loss.item(), epoch)
    tensorboard.add_scalar("val/acc", binacc, epoch)

# Train

In [44]:
for e in range(nb_epochs):
    train(e)
    val(e)

KeyboardInterrupt: 

# ♫♪.ılılıll|̲̅̅●̲̅̅|̲̅̅=̲̅̅|̲̅̅●̲̅̅|llılılı.♫♪